# Installing Library PYODBC for accessing SQL from Python

In [1]:
pip install pyodbc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Step 1 Create 2 Tables in SQL SERVER and Add common data to both simultanoeusly

In [18]:
import pandas as pd
from sqlalchemy import create_engine

# Replace these values with your SQL Server details
server = 'LAPTOP-LD76VUNJ\SQLEXPRESS'
database = 'Demo'
username = ''
password = ''

# CSV file path
csv_file_path = 'D:\Database_Python\Sql_first21.xlsx'

# Read the CSV file and get the headers
csv_data = pd.read_excel(csv_file_path)

table_name1 = "Table3"  # Replace with your desired table name
table_name2 = "Table4"

# Establish a connection to the SQL Server using SQLAlchemy
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server', pool_size=10, max_overflow=20)
# Insert data into Table 1
csv_data.to_sql(table_name1, con=engine, if_exists='append', index=False)
    
# Insert data into Table 2
csv_data.to_sql(table_name2, con=engine, if_exists='append', index=False)
print("Records Inserted First Time in both Tables")

Records Inserted First Time in both Tables


## Step 2 -- Now Check for duplicates in table 2 with respect to Table 1 If have duplicates delete that common rows from table 1 and fetch non duplicated records from table1 and insert it into table 2.[This is the logic used]

### NOTE: Here we kept the Commented code of Alter Table it is very important to run first if we get errors like Programming Error : Cannot convert nvarchar to float.Run once then again comment .You won't get errors again.

In [17]:
import pandas as pd
from sqlalchemy import create_engine

# Replace these values with your SQL Server details
server = 'LAPTOP-LD76VUNJ\SQLEXPRESS'
database = 'Demo'
username = ''
password = ''

# CSV file path
csv_file_path1 = 'D:\Database_Python\ChargesOUTPUT.xlsx'

# Read the CSV file and get the headers
csv_data = pd.read_excel(csv_file_path)

table_name1 = "Table3"  # Replace with your desired table name
table_name2 = "Table4"

# checking for all Numeric and float columns and creating a list

list_of_int_val=[]
for i in csv_data:
    if (csv_data[i].dtypes == 'int64' or csv_data[i].dtypes == 'float'):
         list_of_int_val.append(i)            
# print(list_of_int_val)     

# fill all Nan values with 0
csv_data = csv_data.fillna('0')
# checking for isnull 
# print(csv_data.isnull().sum())

# converting all 
for i in list_of_int_val:
    csv_data[i] = csv_data[i].astype(str)
print()  


# Establish a connection to the SQL Server using SQLAlchemy
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server', pool_size=10, max_overflow=20)

c = int(input("Press 1 to insert record Temp1 and checking for duplicates in Temp2"))
if c == 1:
    try:
#         convert_numeric_to_str = f'''
#                 ALTER TABLE {table_name1} ALTER COLUMN [VISIT_NUMBER] nvarchar(1500)
#                 ALTER TABLE {table_name1} ALTER COLUMN [CHARGES] nvarchar(1500)
#                 ALTER TABLE {table_name1} ALTER COLUMN [OFFICE_KEY] nvarchar(1500)
#                 ALTER TABLE {table_name1} ALTER COLUMN [ICD-10_CODE_3] nvarchar(1500)
#                 ALTER TABLE {table_name1} ALTER COLUMN [ICD-10_DESCRIPTION_3] nvarchar(1500)
#                 ALTER TABLE {table_name1} ALTER COLUMN [UNITS] nvarchar(1500)
#                 ALTER TABLE {table_name1} ALTER COLUMN [XX] nvarchar(1500)
#                 ALTER TABLE {table_name1} ALTER COLUMN [YY] nvarchar(1500)
#                 ALTER TABLE {table_name1} ALTER COLUMN [TRANSACTION_CARRIER_NAME] nvarchar(1500)
#                 ALTER TABLE {table_name1} ALTER COLUMN [CBO] nvarchar(1500)
 
#                 ALTER TABLE {table_name2} ALTER COLUMN [VISIT_NUMBER] nvarchar(1500)
#                 ALTER TABLE {table_name2} ALTER COLUMN [CHARGES] nvarchar(1500)
#                 ALTER TABLE {table_name2} ALTER COLUMN [OFFICE_KEY] nvarchar(1500)
#                 ALTER TABLE {table_name2} ALTER COLUMN [ICD-10_CODE_3] nvarchar(1500)
#                 ALTER TABLE {table_name2} ALTER COLUMN [ICD-10_DESCRIPTION_3] nvarchar(1500)
#                 ALTER TABLE {table_name2} ALTER COLUMN [UNITS] nvarchar(1500)
#                 ALTER TABLE {table_name2} ALTER COLUMN [XX] nvarchar(1500)
#                 ALTER TABLE {table_name2} ALTER COLUMN [YY] nvarchar(1500)
#                 ALTER TABLE {table_name2} ALTER COLUMN [TRANSACTION_CARRIER_NAME] nvarchar(1500)
#                 ALTER TABLE {table_name2} ALTER COLUMN [CBO] nvarchar(1500)
#         '''
#         alter_rows = pd.read_sql_query(convert_numeric_to_str, engine)
        
#         print("Rows DataType Altered")
        
        # Insert data into Table 1
        csv_data.to_sql(table_name1, con=engine, if_exists='append', index=False)

        # Check for common rows
        query = f'''
        SELECT *
        FROM {table_name1} t1
        INNER JOIN {table_name2} t2
        ON  t1.[SERVICE_DATE] = t2.[SERVICE_DATE] And
        t1.[ENTRY_DATE] = t2.[ENTRY_DATE] and
        t1.[VISIT_NUMBER] = t2.[VISIT_NUMBER] and
        t1.[CPT_CODE] = t2.[CPT_CODE] and
        t1.[GROUP_CODE] = t2.[GROUP_CODE] and
        t1.[ENTRY_YEAR_AND_MONTH] = t2.[ENTRY_YEAR_AND_MONTH] and
        t1.[CHARGES] = t2.[CHARGES] and
        t1.[VOID]=t2.[VOID];
        '''

        common_rows = pd.read_sql_query(query, engine)

        if not common_rows.empty:
            print("Common rows found:")
#             print(common_rows)
            print("common rows length:",len(common_rows))
            
            # Execute an SQL query to delete common records from table1
            delete_common_records_query = f'''
            DELETE FROM {table_name1}
            WHERE EXISTS (
                SELECT 1
                FROM {table_name2} t2
                WHERE
                    {table_name1}.[SERVICE_DATE] = t2.[SERVICE_DATE] And
                    {table_name1}.[ENTRY_DATE] = t2.[ENTRY_DATE] and
                    {table_name1}.[VISIT_NUMBER] = t2.[VISIT_NUMBER] and
                    {table_name1}.[CPT_CODE] = t2.[CPT_CODE] and
                    {table_name1}.[GROUP_CODE] = t2.[GROUP_CODE] and
                    {table_name1}.[ENTRY_YEAR_AND_MONTH] = t2.[ENTRY_YEAR_AND_MONTH] and
                    {table_name1}.[CHARGES] = t2.[CHARGES] and
                    {table_name1}.[VOID]=t2.[VOID]
                    )
            '''

            # Execute the DELETE query without trying to fetch results
            engine.execute(delete_common_records_query)
            print("Deleted common rows.")

            # Fetch remaining rows from Temp1
            remaining_rows_query = f'''
            SELECT * FROM {table_name1}
            '''
            remaining_rows = pd.read_sql_query(remaining_rows_query, engine)
#             print(remaining_rows)
            print("Table 1 has remaining rows length:",len(remaining_rows))
            
            if not remaining_rows.empty:
                print("Remaining rows in Temp1:")
              
                remaining_rows.to_sql(table_name2, con=engine, if_exists='append', index=False)
                # Fetch remaining rows from Temp2
                remaining_rows_query2 = f'''
                SELECT * FROM {table_name2}
                '''
                remaining_rows2 = pd.read_sql_query(remaining_rows_query2, engine)
                print("Table 1 has remaining rows length:",len(remaining_rows2))
                print("Inserted remaining rows into Temp2.")
            else:
                print("No remaining rows in Temp1.")

        else:
            try:
                print("No common rows.")
                csv_data.to_sql(table_name2, con=engine, if_exists='append', index=False)
                print(" Data Inserted into Table2")
                
            except:
#                 print(f"Error converting column {col}: {str(e)}")
                  pass  
                
    except Exception as e:
        print("Error:", str(e))
       


Press 1 to insert record Temp1 and checking for duplicates in Temp21
Common rows found:
common rows length: 20
Deleted common rows.
Table 1 has remaining rows length: 0
No remaining rows in Temp1.


In [14]:
import pandas as pd
from sqlalchemy import create_engine

# Replace these values with your SQL Server details
server = 'LAPTOP-LD76VUNJ\SQLEXPRESS'
database = 'Demo'
username = ''
password = ''

# CSV file path
csv_file_path = 'D:\Database_Python\Sql_first21.xlsx'

# Read the CSV file and get the headers
csv_data = pd.read_excel(csv_file_path)

table_name1 = "Temp1"  # Replace with your desired table name
table_name2 = "Temp2"
patient_details = "Patient_Details"
# Establish a connection to the SQL Server using SQLAlchemy
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server', pool_size=10, max_overflow=20)

c = int(input("Press 1 to insert record Temp1 and checking for duplicates in Temp2"))
if c == 1:
    try:
 
        # Check if the table exists
        table_exists_query = f"IF OBJECT_ID('{patient_details}', 'U') IS NOT NULL SELECT 1 ELSE SELECT 0"
        result = engine.execute(table_exists_query).scalar()

        if result:
            print(f"Table '{patient_details}' exists")
            select_patient_data = f''' SELECT * FROM {patient_details} '''
            fetch_patient_details = pd.read_sql_query(select_patient_data, engine)
            
            if not fetch_patient_details.empty:
               # Check for common rows
                    query = f'''
                    SELECT *
                    FROM {table_name1} t1
                    INNER JOIN {patient_details} t2
                    ON  t1.[PATIENT_NAME_FIRST_LAST] = t2.[PATIENT_NAME_FIRST_LAST];
                   '''
                    common_rows = pd.read_sql_query(query, engine)
                    if not common_rows.empty:
                        print("Common rows found:")
                        print(common_rows)
                        print("common rows length:",len(common_rows))
                    else:
                        print("No common Rows Found:")
                
        else:
            print(f"Table '{patient_details}' does not exist")    
            
            create_patient_table_query = f'''CREATE TABLE {patient_details} 
                                                (
                                                  Id INT IDENTITY(1,1) PRIMARY KEY,
                                                  [PATIENT_NAME_(FIRST_LAST)] VARCHAR(MAX)
                                                 );
                                            '''
            created_patient = pd.read_sql_query(create_patient_table_query, engine)        
            insert_patient_data = f'''SELECT DISTINCT [PATIENT_NAME_(FIRST_LAST)] FROM {table_name1}'''
            insert_patient_details = pd.read_sql_query(insert_patient_data, engine)
            insert_patient_details.to_sql(patient_details, con=engine, if_exists='append', index=False)
            print("Patient Details Table Created and Data Inserted Successfully")
                
    except Exception as e:
#             print("Error:", str(e))
            pass
else:
    print("Problem in table creation")
    


Press 1 to insert record Temp1 and checking for duplicates in Temp21


In [11]:
insert_patient_data = f'''SELECT DISTINCT [PATIENT_NAME_(FIRST_LAST)] FROM {table_name1}'''
insert_patient_details = pd.read_sql_query(insert_patient_data, engine)
insert_patient_details.to_sql(patient_details, con=engine, if_exists='append', index=False)
print("Patient Details Table Created and Data Inserted Successfully")
                

Patient Details Table Created and Data Inserted Successfully
